In [1]:
%load_ext autoreload
%autoreload 2

Dependencies

In [2]:
import pathlib as pb

# Environment
ROOT_PATH = pb.Path('..')
DATA_DIR_PATH = ROOT_PATH / 'data'
CACHE_DIR_PATH = ROOT_PATH / '.cache'
TRANSFORMERS_CACHE_DIR_PATH = CACHE_DIR_PATH / 'transformers'
DATASETS_CACHE_DIR_PATH = CACHE_DIR_PATH / 'datasets'
TEST_DATA_FILE = DATA_DIR_PATH / 'test_data.csv'
TRAIN_DATA_FILE = DATA_DIR_PATH / 'train_data.csv'

# Model Repositories
BERT_RO_MODEL_REPO = 'dumitrescustefan/bert-base-romanian-cased-v1'

In [3]:

import os

os.environ['TRANSFORMERS_CACHE'] = str(TRANSFORMERS_CACHE_DIR_PATH)
os.environ['HF_DATASETS_CACHE'] = str(DATASETS_CACHE_DIR_PATH)
os.environ['TOKENIZERS_PARALLELISM'] = 'true'

In [4]:
from transformers import AutoTokenizer, AutoModel
import torch
import torchdata
import torchtext
from torch import Tensor
from torch import nn
from torch import optim
from torch.utils.data import Subset, DataLoader, Dataset
from torch import backends
import typing
import pathlib as pb
import os
import gc
from typing import List, Tuple, Dict, Set
import random
import numpy as np
import pandas as pd
import platform
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import KFold
from sklearn.metrics import balanced_accuracy_score

/home/invokariman/projects/git/nitro-lang-processing-2/.cache/pypoetry/virtualenvs/nitro-lang-processing-2-NQx-kwHa-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from utils import get_available_device, read_data
from preprocess import BertPreprocessor
from data import SexismDataset
from models import BertFlatClassModel

Environment Setup

In [6]:
# Use available GPU
DEVICE: torch.device = get_available_device()

# Deterministic experiments
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
np.random.RandomState(SEED)
torch.manual_seed(SEED)
torch.use_deterministic_algorithms(True)

Dataset & Preprocessing

In [7]:
train_data_raw, test_data_raw = read_data(DATA_DIR_PATH)

In [8]:
train_dataset = SexismDataset(train_data_raw, BertPreprocessor(train_data_raw, BERT_RO_MODEL_REPO))
test_dataset = SexismDataset(test_data_raw, BertPreprocessor(test_data_raw, BERT_RO_MODEL_REPO))

Model Training

In [9]:
# Training specifications
batch_size = 8
num_epochs = 10
num_workers = 6
learning_rate = 1e-5

In [10]:
# Instantiate language model
# Inspired from: https://luv-bansal.medium.com/fine-tuning-bert-for-text-classification-in-pytorch-503d97342db2
model = BertFlatClassModel(BERT_RO_MODEL_REPO)
loss_fn = nn.CrossEntropyLoss() # TODO: weighted loss
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

Some weights of the model checkpoint at dumitrescustefan/bert-base-romanian-cased-v1 were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
# Inspired from: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html
kf = KFold(n_splits=5, shuffle=True, random_state=SEED)

for i, (train_idx, valid_idx) in enumerate(kf.split(train_dataset)):
    # Split the data
    train_subset = Subset(train_dataset, train_idx)
    valid_subset = Subset(train_dataset, valid_idx)

    # Create the dataloaders
    train_loader = DataLoader(
        train_subset,
        batch_size,
        pin_memory=True,
        num_workers=num_workers
    )
    valid_loader = DataLoader(
        train_subset,
        batch_size,
        pin_memory=True,
        num_workers=num_workers
    )

    # ---  Training  ---
    for epoch in range(num_epochs):
        epoch_loss = []
        epoch_accy = []

        for batch, X in train_loader:
            # Send batch to GPU
            y_true = X['label'].to(DEVICE)
            input_ids = X['input_ids'].to(DEVICE)
            attention_mask = X['attention_mask'].to(DEVICE)

            # Make predictions
            y_pred = model.forward({
                'input_ids': input_ids,
                'attention_mask': attention_mask
            })

            # Compute the loss
            optimizer.zero_grad()
            loss: Tensor = loss_fn(y_pred, y_true)
            loss.backward()
            optimizer.step()

            # Compute the accuracy
            y_pred = torch.argmax(y_pred, dim=1).detach().cpu().numpy()
            epoch_accy.append(balanced_accuracy_score(y_true, y_pred))

            # Track progress
            epoch_loss.append(loss.detach().cpu().numpy())

        mean_loss = np.array(epoch_loss).mean()
        mean_accy = np.array(epoch_accy).mean()
        print('Epoch {} - Loss: {}, Accuracy: {}'.format(epoch, mean_loss, mean_accy))

    # --- Validation ---

In [ ]:
# Free memory
del model
gc.collect()
torch.cuda.empty_cache()